In [1]:
# Beta release 1.0
# see Experiment 21356 for detailed explanation, comments, as well as links to github build

In [2]:
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import time

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.base import clone

In [3]:
# Model Setup
# This should not be modified

In [4]:
def lambdaFunc(v):
    return int(v)

def openAndParse(f):
    listValue = []
    headerLine = ''
    start = True
    with open(f) as fOpen:
        for i in fOpen:
            if start:
                iLine = i.split(',')
                iLine = iLine.pop(0)
                headerLine = ','.join(iLine)
                start = False
            else:
                i = i.rstrip('\n')
                iSplit = i.split(',')
                iSplitInt = list(map(lambdaFunc, iSplit[1:]))
                # listValue.append((iSplit[0], iSplit[1:]))
                listValue.append((iSplit[0], iSplitInt))
    return (headerLine, listValue)
# ToDo:
# compile single training file
# write parser for test function, include (even though not needed)
# for values with n <= 3, use the following algorithm to generate pseudoreplicates:
# # if sample is a paired sample, for example A001-A003:
# # let A == condition1, B == condition2
# # use sample S with A || (A && X) where X is another condition, where both conditions in S have n > 3
# # use similarities for sub conditions to establish a "new" condition, for example:
# # if sample S1 == A && C, with 1,2,3 ; S2 == A && D, with 1,3,4:
# # # the "new" condition for A would be 1,3, AND create a new replicate of A that already exists
trainingValues = openAndParse('allele_df-trainingSet-HapA.csv')
testingValues = openAndParse('allele_df-testingSet-HapA.csv')

In [5]:
# ToDo:
# Create parser for pivot file that converts input to one-hot vector
# note: re-use existing parser

In [6]:
def valuesToTestList(valList, npArrayTupleList):
    r = []
    matched = False
    for v in valList:
        for vTuple in npArrayTupleList:
            if v == vTuple[0]:
                matched = True
                r.append(vTuple[1])
                break
        if matched:
            matched = False
            continue
        else:
            print('WARNING!')
            print(valList)
            print(npArrayTupleList)
            return None
    return r
        
    
def valuesToIntList(valList):
    listedValues = []
    ct = 0
    l_strings = []
    l_ints = []
    l_strings_r = []
    l_ints_r = []
    npArrayList = []
    for i in valList:
        if i not in l_strings:
            l_strings_r.append(i)
            l_ints_r.append(ct)
            l_strings.append(i)
            l_ints.append(ct)
            npArrayList.append(ct)
            ct += 1
        else:
            v_i = l_strings.index(i)
            v_i_string = l_strings[v_i]
            v_ct = l_ints[v_i]
            l_strings_r.append(v_i_string)
            l_ints_r.append(v_ct)
            npArrayList.append(v_ct)
    npArrayTupleList = list(zip(l_strings_r,l_ints_r))
    return (npArrayList, npArrayTupleList)

def parseAndAddToList(i1,tupleList):
    for itm in tupleList:
        if i1 == itm[0]:
            return itm[1]
    return None


In [7]:
X_train = np.empty([len(trainingValues[1]), len(trainingValues[1][1][1])], dtype=int)
Y_train = np.empty([len(trainingValues[1])], dtype=int)
X_test = np.empty([len(testingValues[1]), len(testingValues[1][1][1])], dtype=int)
Y_test = np.empty([len(testingValues[1])], dtype=int)

trainingValueFromTuple = trainingValues[1]
testingValueFromTuple = testingValues[1]

lookupTableBecauseNumpy = []
lookupTableBecauseNumpyTesting = []
stopRange = len(trainingValueFromTuple)
for x in range(0, stopRange):
    aValue_x = trainingValueFromTuple[x][1]
    aValue_y = trainingValueFromTuple[x][0]
    X_train[x] = aValue_x
    lookupTableBecauseNumpy.append(aValue_y)
    Y_train[x] = np.array(x, dtype=int)
stopRange = len(testingValueFromTuple)
for x in range(0, stopRange):
    aValue_x = testingValueFromTuple[x][1]
    aValue_y = testingValueFromTuple[x][0]
    X_test[x] = aValue_x
    lookupTableBecauseNumpyTesting.append(aValue_y)
    Y_test[x] = np.array(x, dtype=int)

In [22]:
# ToDo:
# create output file from CSV file that matches pivotFile output
X_train


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
parsedTrainingResults = valuesToIntList(lookupTableBecauseNumpy)
Y_train = np.array(parsedTrainingResults[0])
parsedLabelLookupTable = []
for itm in lookupTableBecauseNumpyTesting:
    res = parseAndAddToList(itm, parsedTrainingResults[1])
    if res is not None:
        parsedLabelLookupTable.append((itm, res))
    else:
        print('WARNING!  Check Value ' + str(itm))
Y_testList = valuesToTestList(lookupTableBecauseNumpyTesting, parsedLabelLookupTable)
# print(Y_train)
# print(parsedTrainingResults[1])
Y_test = np.array(Y_testList)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_jobs=-1, weights='distance', n_neighbors=4)
knn_clf.fit(X_train, Y_train)
y_knn_pred = knn_clf.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_knn_pred)

1.0

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(X_train, Y_train)
forest_clf_pred = forest_clf.predict(X_test)

accuracy_score(Y_test, forest_clf_pred)

1.0

In [19]:
cross_val_score(forest_clf, X_train, Y_train, cv=2, scoring="accuracy")

array([ 0.80434783,  0.90243902])